In [1]:
DATA_NAME = 'coco-outdoor' 
TRANSFORM = 'wavelet-horizontal'
CHANNEL = 'green'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_green_horizontal_wavelet_outdoor_coco.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9
2,57442.61,-0.10,-0.05,-0.12,-0.05,-0.00,-0.01,0.00
3,-0.10,10690.23,-0.02,0.03,0.01,0.01,0.00,-0.00
4,-0.05,-0.02,1828.41,-0.03,-0.00,0.00,-0.00,0.00
5,-0.12,0.03,-0.03,298.97,0.00,-0.00,0.00,-0.00
6,-0.05,0.01,-0.00,0.00,47.31,-0.00,0.00,0.00
7,-0.00,0.01,0.00,-0.00,-0.00,7.15,-0.00,-0.00
8,-0.01,0.00,-0.00,0.00,0.00,-0.00,1.07,0.00
9,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,0.14


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9
2,1.00000,-0.00000,-0.00001,-0.00003,-0.00003,-0.00001,-0.00002,0.00001
3,-0.00000,1.00000,-0.00000,0.00002,0.00001,0.00002,0.00000,-0.00001
4,-0.00001,-0.00000,1.00000,-0.00004,-0.00001,0.00004,-0.00002,0.00001
5,-0.00003,0.00002,-0.00004,1.00000,0.00002,-0.00000,0.00001,-0.00002
6,-0.00003,0.00001,-0.00001,0.00002,1.00000,-0.00001,0.00002,0.00003
7,-0.00001,0.00002,0.00004,-0.00000,-0.00001,1.00000,-0.00001,-0.00001
8,-0.00002,0.00000,-0.00002,0.00001,0.00002,-0.00001,1.00000,0.00001
9,0.00001,-0.00001,0.00001,-0.00002,0.00003,-0.00001,0.00001,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.2522895422940417

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[4.12767841e+08 1.39954656e+07 3.98098073e+05 1.02204507e+04
 2.40097618e+02 4.88047362e+00 8.24733224e-02 9.20799058e-34]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7
0,0.999611,-0.027506,-0.004524,-0.000741,-0.000118,-0.000018,-0.000003,-3.438916e-07
1,0.027357,0.999180,-0.029463,-0.004654,-0.000736,-0.000111,-0.000017,-2.279819e-06
2,0.005298,0.029151,0.998978,-0.033716,-0.005168,-0.000777,-0.000117,-1.555100e-05
3,0.001039,0.005574,0.033320,0.998583,-0.040669,-0.005957,-0.000889,-1.199248e-04
4,0.000206,0.001100,0.006441,0.040102,0.997812,-0.051607,-0.007478,-1.002355e-03
5,0.000041,0.000220,0.001287,0.007887,0.050592,0.995725,-0.076230,-9.903538e-03
6,0.000009,0.000049,0.000285,0.001742,0.011013,0.073740,0.988140,-1.342262e-01
7,0.000002,0.000013,0.000078,0.000476,0.003002,0.019888,0.133083,9.909007e-01


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.00038887219997640887,
 0.0008196838219430846,
 0.0010215094178773754,
 0.001417319638364778,
 0.002187966152456755,
 0.004275409951461029,
 0.011859681923872878,
 0.009099291139358368]